# Domain Definiftions

In [25]:
from dataclasses import dataclass
from enum import Enum
from typing import List, Dict, Tuple, Optional


In [26]:
@dataclass(frozen=False)
class Node:
    x: int
    y: int
    blocked: bool

    def __init__(self, x: int, y: int):
        self.x = x
        self.y = y
        self.blocked = Node.is_blocked()

    """
    Generate a binomial random distribution of '0' and '1's
    where '0's represent the node is blocked, and therefore 
    it cannot be reached
    """
    @classmethod
    def is_blocked(cls):
        dist = np.random.binomial(1, 0.8)
        return True if dist == 0 else False

    @property
    def id(self) -> Tuple[int, int]:
        return self.x, self.y

    def is_free(self):
        return not self.blocked

    def find_neighbors(self):
        pass

    def __hash__(self):
        return self.id.__hash__()

In [27]:
class Terrain(Enum):
    Asphalt = ("Terra", 1)
    Flooding = ("Agua", 3)
    Quicksand = ("Areia Movedica", 6)


In [28]:
@dataclass(frozen=False)
class Edge:
    source: Node
    destination: Node
    terrain: Terrain

    def __init__(self, source: Node, dest: Node, terrain: Terrain = Terrain.Asphalt):
        self.source = source
        self.destination = dest
        self.terrain = terrain

    @property
    def weight(self):
        return self.terrain.value[1]
    
    def update_terrain(self, terrain: Terrain):
        self.terrain = terrain

In [29]:
@dataclass(frozen=False)
class Graph:
    nodes: Dict[Tuple[int,int], Node]
    edges: Dict[Tuple[Node,Node], Edge]

    def __init__(self):
        self.nodes = dict()
        self.edges = dict()

    def find_neighbors_of(self, node: Node):
        if node.is_free():
            for pos in self._find_neighbors_pos_estimation(node):
                neighbor: Optional[Node] = self.nodes.get(pos)
                if neighbor != None and neighbor.is_free():
                    yield self.nodes.get(pos)

    def find_edge_connecting(self, source: Node, dest: Node):
        return self.edges.get((source, dest))

    """
    estimated positions return a list of expected neighbors as follows: 
    up, down, left, right, up_right, up_left, down_right, down_left
    """
    def _find_neighbors_pos_estimation(self, node: Node):
        pos = node.id
        estimated_neighbors = [
            (pos[0], pos[1]-1),
            (pos[0], pos[1]+1),
            (pos[0]-1, pos[1]),
            (pos[0]+1, pos[1]),
            (pos[0]+1, pos[1]-1),
            (pos[0]-1, pos[1]-1),
            (pos[0]+1, pos[1]+1),
            (pos[0]-1, pos[1]+1)
        ]
        return estimated_neighbors

    def with_node(self, node: Node):
        self.nodes[node.id] = node
        return self

    def with_nodes(self, nodes: List[Node]):
        [self.with_node(node) for node in nodes]
        return self

    def with_edge(self, edge: Edge):
        src = edge.source
        dest = edge.destination
        self.edges[src,dest] = edge
        return self

    def with_edges(self, edges: List[Edge]):
        [self.with_edge(edge) for edge in edges]
        return self

    def reset(self):
        return Graph()


# Graph Construction

### Building the graph

In [41]:
import numpy as np
import matplotlib.pyplot as pt
import math  


In [42]:
def generate_nodes(width: int, height: int):
    for row_num in range(height):
        for col_num in range(width):
            yield Node(row_num, col_num)

In [43]:
area_width = 8
area_height = 8

In [44]:
nodes = generate_nodes(area_width, area_height)
graph = Graph().with_nodes(nodes)

In [45]:
for node in graph.nodes.values():
    neighbors: List[Node] = graph.find_neighbors_of(node)
    edges: List[Edge] = [Edge(node, neighbor) for neighbor in neighbors]
    graph.with_edges(edges)

### Sort Terrains in Graph to screw with the edges originating from it

# PathFinder Algorithm

### A* Implementation

In [53]:
class AStarAlgorithm:
    graph: Graph

    def __init__(self, graph: Graph):
        self.graph = graph

    def shortest_path_between(self, source: Node, target: Node) -> List[Node]:
        neighbors: List[Node] = graph.find_neighbors_of(source)
        edges: List[Edge] = [graph.find_edge_connecting(source, neighbor) for neighbor in neighbors]
        heuristics: List[float] = [a_star.distance_between(edge.destination, target, edge.weight) for edge in edges]
        return heuristics

    def distance_between(self, node: Node, target: Node, cost_to_node: int):
        dist = pow(target.x - node.x,2) + pow(target.y - node.y,2)
        return math.sqrt(dist) + cost_to_node

In [54]:
a_star = AStarAlgorithm(graph)
source = graph.nodes.get((0,0))
dest = graph.nodes.get((area_width-1, area_height-1))

In [55]:
path: List[Node] = a_star.shortest_path_between(source, dest)

In [56]:
path

[10.219544457292887, 10.219544457292887, 9.48528137423857]

# Graph Visualization